In [ ]:
import os
import pandas as pd
import sys
import urllib.request
import time
import glob

In [ ]:
add_continuous_traits = True

In [ ]:
pheno_data_dir = "/u/project/sriram/ukbiobank/33127/"

In [ ]:
pheno_files = glob.glob(os.path.join(pheno_data_dir, "ukb*.csv"))

In [ ]:
pheno_files

In [ ]:
phenotype_file = pheno_files[0]
#html_file = os.path.join(pheno_data_dir, os.path.splitext(os.path.basename(phenotype_file))[0] + ".html")
html_file = os.path.join(pheno_data_dir, (os.path.splitext(os.path.basename(phenotype_file))[0]).split(".")[0] + ".html")

ukbb_phenotype_code_url = "http://biobank.ndph.ox.ac.uk/showcase/coding.cgi?id={}"

In [ ]:
print("Phenotype file: {}".format(phenotype_file))
print("HTML file: {}".format(html_file))

# Read in HTML file with phenotype mapping

In [ ]:
html_text = ""
encoding = None
try:
    with open(html_file, "r", encoding=encoding) as hf:
        html_text = hf.read()
except UnicodeDecodeError:
    encoding = "latin-1"
    with open(html_file, "r", encoding=encoding) as hf:
        html_text = hf.read()
print("final encoding: {}".format(encoding))

In [ ]:
len(html_text)

## Create pandas data frame from html table

The .html file has the mapping from phenotype -> .csv column index, so we can search this .html file for specific phenotypes of interest, figure out which column indices we need, and then use these indices to index the large .csv file which contains the data per patient. 

In [ ]:
start_time = time.time()
html_df = pd.read_html(html_text, encoding=encoding)[1]
print("elapsed time: {}".format(time.time() - start_time))

In [ ]:
html_df.head()

In [ ]:
# create column that contains codes for mapping categorical variable integers to strings
def find_codes(description):
    term1 = "Uses data-coding"
    term2 = "comprises"
    if term1 in description:
        code = description.split(term1)[1].strip()
        if term2 in description: 
            code = code.split(term2)[0].strip()
        return code
    else:
        return None
        
html_df["Data_Coding"] = html_df["Description"].apply(find_codes)
html_df["Data_Coding"]

In [ ]:
# Remove code from phenotype description col
def remove_codes(description):
    term = "Uses data-coding"
    if term in description:
        return description.split(term)[0].strip()
    else:
        return description

In [ ]:
html_df["Description"] = html_df["Description"].apply(remove_codes)
html_df["Description"]

## Search for rows matching a particular term

In [ ]:
search_terms = [
    "BMI",
    "Standing height",
    "Weight",
    "Body fat percentage",
    "Pulse rate",
    "Systolic blood pressure",
    "Diastolic blood pressure",
    "Vascular/heart problems diagnosed by doctor",
    "Age stroke diagnosed",
    "Age angina diagnosed",
    "Age heart attack diagnosed",
    "Age high blood pressure diagnosed",
    "Diabetes diagnosed by doctor",
    "Medication for cholesterol, blood pressure or",
    "Frequency of drinking alcohol",
    "Alcohol intake frequency",
    "Smoking status",
    "Qualifications",
    "Job code",
    "Average total household income before tax",
    "Townsend",
    "Number of live births",
    "Age completed full time education",
    "Age when attended assessment centre",
    "UK Biobank assessment centre",
    "Sex",
    "Genetic sex",
    "Genotype measurement batch",
    "Genotype measurement plate",
    "Genotype measurement well",
    "Genetic kinship to other participants",
    "Date of attending assessment centre",
    "Ethnic background",
    "Genetic principal components",
    "Outliers for heterozygosity or missing rate"
]

In [ ]:
rows_to_use = []
missing_features = []
print("Querying for {} terms".format(len(search_terms)))
for term in search_terms:
    # look for all rows that contain search term
    rows = html_df[html_df["Description"].str.contains(term)].sort_values("Count", ascending=False)
    # if we don't find any, add to list of missing features
    if rows.shape[0] == 0:
        missing_features.append(term)
    # otherwise add these rows for use
    else:
        rows_to_use.append(rows)
        
print("Found {} rows".format(len(rows_to_use)))
print("Could not find {} terms".format(len(missing_features)))

In [ ]:
print("Missing terms:\n")
[print(f) for f in missing_features]

In [ ]:
# if looking to dump all continuous traits, set this flag to true
if add_continuous_traits:
    rows = html_df[html_df["Type"] == "Continuous"]
    print("Adding {} rows for continuous traits".format(rows.shape[0]))
    rows_to_use.append(rows)

In [ ]:
chd = html_df[html_df["Description"].str.contains("Alcohol")].sort_values("Count", ascending=False)
chd

In [ ]:
# merge all rows to use into single DF
relevant_rows = pd.concat(rows_to_use)
relevant_rows

# Read in phenotype file

The phenotype file is large, so we can save time/memory by only reading the columns we specifically need. We can use the phenotype -> column index mapping from the .html file to only grab specific columns from the .csv file. 

In [ ]:
# select only relevant columns 
relevant_cols = relevant_rows["Column"]
relevant_cols

In [ ]:
# we add in column 0 since it contains the EID
start_time = time.time()
pheno_df = pd.read_csv(phenotype_file, sep=",", header=0, usecols=[0]+list(relevant_cols))
print("Loading took {} seconds".format(time.time() - start_time))

In [ ]:
pheno_df.head()

## Map categorical phenotype integers to strings 

In [ ]:
for col in pheno_df.columns.values:
    pheno_row = html_df[html_df["UDI"] == col]
    
    # if we have a code in the Data_Coding column, go get mapping
    code = pheno_row["Data_Coding"].iloc[0]
    if code is not None:
        # get contents of UKBB webpage, and include code in URL
        
        try:
            contents = urllib.request.urlopen(ukbb_phenotype_code_url.format(code)).read()
            try:
                # convert returned HTML into pandas data frame
                mapping_df = pd.read_html(contents)[1]
                # set Coding column as index
                mapping_df.set_index("Coding", inplace=True)
                print(mapping_df)
                # create a mapping from integer codes to string codes
                mapping_dict = mapping_df.to_dict(orient="dict")
                # replace integer coding with string coding
                pheno_df[col].replace(mapping_dict["Meaning"], inplace=True)

            # found to happen when there is a hierarchical tree-structured dict mapping which 
            # requires more work...
            except IndexError as e:
                pass
        except:
            print("ERROR: cannot make URL Request")
            print("Row:", pheno_row)
            print("Col:", col)
            print("Code:", code)
        
        
        

In [ ]:
pheno_df.describe(include="all")

## Rename columns using phenotype description

In [ ]:
# rename columns to phenotype description
column_mapping = {}
for col in pheno_df.columns.values:
    column_mapping[col] = html_df[html_df["UDI"] == col]["Description"].values[0] + "_" + col

column_mapping

In [ ]:
pheno_df.rename(columns=column_mapping, inplace=True)

In [ ]:
pheno_df.to_csv("/u/scratch/b/blhill/UKBB_phenotypes_{}.tsv".format(os.path.splitext(os.path.basename(phenotype_file))[0]), 
                sep="\t", header=True, index=False)

In [ ]:
pheno_df = pd.read_csv("UKBB_phenotypes_{}.tsv".format(os.path.splitext(os.path.basename(phenotype_file))[0]), 
                       sep="\t", header=0)

In [ ]:
pheno_df.columns.values

In [ ]:
for c in pheno_df.columns.values:
    print(c, pheno_df[c].dtypes)

# Merge the two .tsv files 

In [ ]:
pheno_df1 = pd.read_csv("/u/scratch/b/blhill/UKBB_phenotypes_ukb39967.enc_ukb.converted2.tsv", header=0, sep="\t")
pheno_df1.shape

In [ ]:
pheno_df2 = pd.read_csv("/u/scratch/b/blhill/UKBB_phenotypes_ukb21970.tsv", header=0, sep="\t")
pheno_df2.shape

In [ ]:
pheno_df1.head()

In [ ]:
pheno_df2.head()

In [ ]:
non_overlapping_cols = list(pheno_df2.columns.values[~pheno_df2.columns.isin(pheno_df1.columns.values)])
non_overlapping_cols = non_overlapping_cols + list(["Encoded anonymised participant ID_eid"])
print("Found {} non-overlapping columns".format(len(non_overlapping_cols)))

In [ ]:
merged_df = pd.merge(pheno_df1, pheno_df2[non_overlapping_cols], how="left", on="Encoded anonymised participant ID_eid")

In [ ]:
print("merged dataframe shape:", merged_df.shape)

In [ ]:
merged_df.head()

In [ ]:
merged_df.to_csv("/u/scratch/b/blhill/UKBB_phenotypes_merged.tsv", header=True, index=False, sep="\t")